In [1]:
import pandas as pd
import numpy as np

import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from numpy import argsort
import joblib
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import lightgbm as lgb
import re
from xgboost import XGBRegressor
from scipy.stats import skew
from sklearn.preprocessing import PowerTransformer
from catboost import CatBoostRegressor, cv, Pool
import warnings
from joblib import Parallel, delayed

warnings.filterwarnings('ignore')
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
pd.set_option('display.max_columns', 1000)
import optuna

df_train = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')

In [2]:
from sklearn.model_selection import train_test_split

num_cols = df_train.select_dtypes(exclude=['object']).columns
cat_cols = df_train.select_dtypes(include=['object']).columns

# LABEL ENCODE CATEGORICAL FEATURES
CAT_SIZE = []
CAT_EMB = []
RARE = []

print("LABEL ENCODING:")
for c in cat_cols:
    # LABEL ENCODE
    df_train[c],_ = df_train[c].factorize()
    df_train[c] -= df_train[c].min()
    vc = df_train[c].value_counts()
    
    # IDENTIFY RARE VALUES
    RARE.append( vc.loc[vc<40].index.values )
    n = df_train[c].nunique()
    mn = df_train[c].min()
    mx = df_train[c].max()
    r = len(RARE[-1])
    print(f'{c}: nunique={n}, min={mn}, max={mx}, rare_ct={r}')
    
    # RELABEL RARE VALUES AS ZERO
    CAT_SIZE.append(mx+1 +1) #ADD ONE FOR RARE
    CAT_EMB.append( int(np.ceil( np.sqrt(mx+1 +1))) ) # ADD ONE FOR RARE
    df_train[c] += 1
    df_train.loc[df_train[c].isin(RARE[-1]),c] = 0 

LABEL ENCODING:
person_home_ownership: nunique=4, min=0, max=3, rare_ct=0
loan_intent: nunique=6, min=0, max=5, rare_ct=0
loan_grade: nunique=7, min=0, max=6, rare_ct=1
cb_person_default_on_file: nunique=2, min=0, max=1, rare_ct=0


In [3]:
X = df_train.iloc[:, 0:10]
y = df_train.iloc[:, 11]

from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y = lab_enc.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=0, 
)

In [4]:
df_test = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')

num_cols = df_test.select_dtypes(exclude=['object']).columns
cat_cols = df_test.select_dtypes(include=['object']).columns

# LABEL ENCODE CATEGORICAL FEATURES
CAT_SIZE = []
CAT_EMB = []
RARE = []

print("LABEL ENCODING:")
for c in cat_cols:
    # LABEL ENCODE
    df_test[c],_ = df_test[c].factorize()
    df_test[c] -= df_test[c].min()
    vc = df_test[c].value_counts()
    
    # IDENTIFY RARE VALUES
    RARE.append( vc.loc[vc<40].index.values )
    n = df_test[c].nunique()
    mn = df_test[c].min()
    mx = df_test[c].max()
    r = len(RARE[-1])
    print(f'{c}: nunique={n}, min={mn}, max={mx}, rare_ct={r}')
    
    # RELABEL RARE VALUES AS ZERO
    CAT_SIZE.append(mx+1 +1) #ADD ONE FOR RARE
    CAT_EMB.append( int(np.ceil( np.sqrt(mx+1 +1))) ) # ADD ONE FOR RARE
    df_test[c] += 1
    df_test.loc[df_test[c].isin(RARE[-1]),c] = 0 

LABEL ENCODING:
person_home_ownership: nunique=4, min=0, max=3, rare_ct=0
loan_intent: nunique=6, min=0, max=5, rare_ct=0
loan_grade: nunique=7, min=0, max=6, rare_ct=1
cb_person_default_on_file: nunique=2, min=0, max=1, rare_ct=0


In [5]:
df_test = df_test.drop(['cb_person_default_on_file', 'cb_person_cred_hist_length'], axis=1)
df_test

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income
0,58645,23,69000,1,3.0,1,1,25000,15.76,0.36
1,58646,26,96000,2,6.0,2,2,10000,12.68,0.10
2,58647,26,30000,1,5.0,3,3,4000,17.19,0.13
3,58648,33,50000,1,4.0,4,4,7000,8.90,0.14
4,58649,26,102000,2,8.0,1,5,15000,16.32,0.15
...,...,...,...,...,...,...,...,...,...,...
39093,97738,22,31200,2,2.0,4,0,3000,10.37,0.10
39094,97739,22,48000,2,6.0,5,4,7000,6.03,0.15
39095,97740,51,60000,2,0.0,2,4,15000,7.51,0.25
39096,97741,22,36000,2,4.0,2,5,14000,15.62,0.39


In [6]:
sample = pd.read_csv('/kaggle/input/playground-series-s4e10/sample_submission.csv')

In [9]:
#load libraries
import pandas as pd
import numpy as np
#from statistics import *
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
#load libraries
import pandas as pd
import numpy as np
#from statistics import *
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier


##### Decision Tree
DecisionTree = DecisionTreeClassifier()
DecisionTree.fit(X_train, y_train)
DecisionTree_pred = DecisionTree.predict(df_test)
#make submission table
sample['loan_status'] = DecisionTree_pred

#save file
sample.to_csv('DecisionTree2.csv', 
                              index = False)

##### KNN
KNN = KNeighborsClassifier(n_neighbors=2)
KNN.fit(X_train, y_train)
KNN_pred = KNN.predict(df_test)
#make submission table
sample['loan_status'] = KNN_pred

#save file
sample.to_csv('KNN2.csv', 
                     index = False)

##### MLPC
MLPC = MLPClassifier(random_state=1, max_iter=300)
MLPC.fit(X_train, y_train)
MLPC_pred = MLPC.predict(df_test)
#make submission table
sample['loan_status'] = MLPC_pred
#save file
sample.to_csv('MLPC2.csv', 
                      index=False)

##### Random Forest
RandomForest = RandomForestClassifier()
RandomForest.fit(X_train,y_train)
RandomForest_pred = RandomForest.predict(df_test)
#make submission table
sample['loan_status'] = RandomForest_pred
#save file
sample.to_csv('RandomForest2.csv', 
                              index=False)

##### XGB Classifier
XGB = XGBClassifier()
XGB.fit(X_train,y_train)
XGB_pred = XGB.predict(df_test)
#make submission table
sample['loan_status'] = XGB_pred

#save file
sample.to_csv('XGB2.csv', 
                     index=False)

In [12]:

##### AVERAGE
average_pred = (XGB_pred+
                KNN_pred+
                MLPC_pred+
                RandomForest_pred+
                DecisionTree_pred)/5

sample['loan_status'] = average_pred
sample.to_csv('avg.csv', index=False)



# ##### WEIGHTED AVERAGE
weighted_average = (0.3*XGB_pred+
                    0.2*DecisionTree_pred+
                    0.2*RandomForest_pred+
                    0.15*MLPC_pred+
                    0.15*KNN_pred)

#make submission table
sample['loan_status'] = weighted_average
sample.to_csv('wgt.csv', index=False)